In [ ]:
!pip install awscli

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [3]:
import os
list_building = ['airport_terminal','burial_site','park','stadium','zoo']

In [8]:
for bui in list_building:
  os.mkdir(bui)



In [11]:
%cd DataSet

/content/DataSet


In [ ]:
%cd airport_terminal
!aws s3 cp s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/train/airport_terminal/ . --recursive --no-sign-request
%cd ..

In [ ]:
%cd burial_site
!aws s3 cp s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/train/burial_site/ . --recursive --no-sign-request
%cd ..

In [ ]:
%cd park
!aws s3 cp s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/train/park/ . --recursive --no-sign-request
%cd ..

In [ ]:
%cd stadium
!aws s3 cp s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/train/stadium/ . --recursive --no-sign-request
%cd ..

In [ ]:
%cd zoo
!aws s3 cp s3://spacenet-dataset/Hosted-Datasets/fmow/fmow-rgb/train/zoo/ . --recursive --no-sign-request
%cd ..

In [17]:
%cd ..

/content


In [27]:
file_ = os.path.join('DataSet','airport_terminal')
file_dest = os.path.join('NewDS','airport_terminal')
#os.listdir(file_)
# dir1 = os.path.join(file_,'airport_terminal_0')
# os.listdir(dir1)

In [19]:
!ls

DataSet  NewDS	sample_data


In [28]:
from PIL import Image
import json

def portal_method(size):
  for sub_file1 in os.listdir(file_):
    dir1 = os.path.join(file_,sub_file1)
    for sub_file2 in os.listdir(dir1):
      dir2 = os.path.join(dir1,sub_file2)    
      if os.path.splitext(dir2)[1] == '.json':
        with open(dir2) as f:
          try:
            data = json.load(f)
          except:
            continue
            reconstruct_from_config
          bbox = data['bounding_boxes'][0]['box']
          img_path = dir2[:-5]
          im_p = img_path + '.jpg'
          new_image = prepare_image(im_p,bbox,size,file_dest)
          os.remove(im_p)
          os.remove(dir2)

def prepare_image(image_path, bbox, size,output_dir):
    """
    Preprocesses an image for input to a CNN.
    
    :param image_path: Path to the input image.
    :type image_path: str
    :param bbox: Bounding box coordinates in the format (x_min, y_min, x_max, y_max).
    :type bbox: tuple
    :param size: Size of the output image in pixels.
    :type size: int
    :return: Preprocessed image as a numpy array.
    """
    # Open the image file
    try:
      image = Image.open(image_path)
    except :
      return
    # Enlarge the bounding box in proportion to its size
    width, height = image.size
    x_min, y_min, x_max, y_max = bbox
    if x_min > x_max or y_min > y_max:
      return
    box_width = x_max - x_min
    box_height = y_max - y_min
    box_size = max(box_width, box_height)
    new_x_min = max(0, x_min - box_size // 2)
    new_y_min = max(0, y_min - box_size // 2)
    new_x_max = min(width, x_max + box_size // 2)
    new_y_max = min(height, y_max + box_size // 2)
    bbox = (new_x_min, new_y_min, new_x_max, new_y_max)
    
    # Crop the image to the bounding box
    image = image.crop(bbox)
    # Resize the image for input to the CNN
    image = image.resize((size, size))
    # Convert the image to a numpy array
    filename = f"preprocessed_{os.path.basename(image_path)}"
    output_path = os.path.join(output_dir, filename)
    image.save(output_path)

portal_method(224)


In [ ]:
#cuidado donde correr esto, asegurarse de estar en el directorio correcto
for fil in os.listdir(file_):
  dir1 = os.path.join(file_,fil)
  print(dir1)
  # for file in os.listdir(dir1):
      # if os.path.splitext(os.path.join(file_dir, file))[1] == '.json':
      #     print(os.path.splitext(os.path.join(file_dir, file))[1])
      # if file.endswith('.jpg') or file.endswith('.json'):
      #     os.remove(os.path.join(dir1,file))
      #     print('delete file,',os.path.join(file_dir, file))
  if os.path.isdir(os.path.join(file_, fil)):
    os.rmdir(os.path.join(file_, fil))
    print('a')

In [53]:
import os
file_dir = 'NewDS'
image_ext = ['jpeg','jpg','bmp','png']
os.listdir(file_dir)


['airport_terminal', 'stadium', 'burial_site', 'park', 'zoo']

In [30]:
for x in os.listdir(file_dir):
  c = 0
  for y in os.listdir(os.path.join(file_dir,x)):
    if c >= 1000:
      os.remove(os.path.join(file_dir,x,y))
    c += 1

In [54]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import json
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [55]:
datagen = ImageDataGenerator(rescale=1./255)
train_dir = 'NewDS'
train_generator= datagen.flow_from_directory(train_dir,target_size=(224,224),class_mode='categorical',batch_size=32)
num_classes = train_generator.num_classes

Found 5000 images belonging to 5 classes.


In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(train_generator.filenames, train_generator.classes, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Crear generadores de datos para los conjuntos de entrenamiento, validación y prueba
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224,224), class_mode='categorical', subset='training')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(train_dir, target_size=(224,224), class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(train_dir, target_size=(224,224), class_mode='categorical', subset='validation')

In [57]:
print(len(X_train),len(X_val),len(X_test))

3200 800 1000


**Model**

In [58]:
pre = tf.keras.metrics.Precision()
acc = tf.keras.metrics.CategoricalAccuracy()
batch_size = 16


In [59]:
from keras.models import Model
from keras.layers import Input, concatenate, Dense, Dropout, Flatten,Reshape
from keras.applications.densenet import DenseNet121
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras import backend 

# Define the number of classes


# Load DenseNet121 model
densenet_model = DenseNet121(weights='imagenet')

# Load ResNet152 model
resnet_model = ResNet152(weights='imagenet')

# # Load InceptionV3 model
#inception_model = InceptionV3(weights='imagenet')

# # Load Xception model
#ception_model = Xception(weights= 'imagenet')

# Define the input shape for all models
input_shape = (224, 224, 3)

# Define the input tensor for all models
input_tensor = Input(shape=input_shape)

# Get the output tensors for all models
densenet_output = densenet_model(input_tensor)
resnet_output = resnet_model(input_tensor)
#inception_output = inception_model(input_tensor)
#xception_output = xception_model(input_tensor)

# Concatenate the outputs of all models
merged = concatenate([densenet_output,resnet_output])

# Add a fully-connected layer with dropout and a logistic layer for the number of classes
x = Dense(1024, activation='relu')(merged)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)


# predicted_class = np.argmax(predictions,axis=1)
# reshape_layer = Reshape((1,))
# predicted_class_reshaped= reshape_layer(predicted_class)

# Define the model with input tensor and output tensor
model = Model(inputs=input_tensor, outputs=predictions)

# Compile the model with categorical cross-entropy loss and Adam optimizer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[acc,pre])


242900224/242900224 [==============================] - 12s 0us/step


In [60]:
logdir = 'logs'

In [61]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [62]:
history_model = model.fit(train_generator,epochs=20,steps_per_epoch=70,validation_data=val_generator,callbacks=[tensorboard_callback])

Epoch 1/20
70/70 [==============================] - 285s 1s/step - loss: 1.4599 - categorical_accuracy: 0.4317 - precision_1: 0.7119
Epoch 2/20
70/70 [==============================] - 77s 1s/step - loss: 1.3358 - categorical_accuracy: 0.4567 - precision_1: 0.6278
Epoch 3/20
70/70 [==============================] - 78s 1s/step - loss: 1.3032 - categorical_accuracy: 0.4670 - precision_1: 0.6755
Epoch 4/20
70/70 [==============================] - 78s 1s/step - loss: 1.1805 - categorical_accuracy: 0.4995 - precision_1: 0.7077
Epoch 5/20
70/70 [==============================] - 78s 1s/step - loss: 1.1476 - categorical_accuracy: 0.5217 - precision_1: 0.7120
Epoch 6/20
70/70 [==============================] - 78s 1s/step - loss: 1.0654 - categorical_accuracy: 0.5549 - precision_1: 0.7172
Epoch 7/20
70/70 [==============================] - 78s 1s/step - loss: 1.0349 - categorical_accuracy: 0.5518 - precision_1: 0.7309
Epoch 8/20
70/70 [==============================] - 78s 1s/step - loss: 0.9

In [63]:
history_model.history

{'loss': [1.459946870803833,
  1.3357717990875244,
  1.303153395652771,
  1.1805256605148315,
  1.1476397514343262,
  1.0654112100601196,
  1.0348947048187256,
  0.9738324880599976,
  0.9475835561752319,
  0.9195767045021057,
  0.9156675934791565,
  0.8593053817749023,
  0.8401759266853333,
  0.8241370916366577,
  0.7698369026184082,
  0.7277987003326416,
  0.7564154267311096,
  0.712100088596344,
  0.641268789768219,
  0.7121441960334778],
 'categorical_accuracy': [0.43169641494750977,
  0.45667868852615356,
  0.4669642746448517,
  0.4995487332344055,
  0.5216606259346008,
  0.5549107193946838,
  0.5517857074737549,
  0.5924107432365417,
  0.6169642806053162,
  0.628125011920929,
  0.6290178298950195,
  0.666967511177063,
  0.684374988079071,
  0.6933035850524902,
  0.7098214030265808,
  0.734375,
  0.7093862891197205,
  0.7472923994064331,
  0.7727678418159485,
  0.7405234575271606],
 'precision_1': [0.7118644118309021,
  0.6278316974639893,
  0.6754966974258423,
  0.7076502442359924

In [ ]:
scores = model.evaluate_generator(test_generator,steps=len(X_test)// 32)

In [ ]:
import matplotlib.pyplot as plt

# Datos de entrenamiento
loss = [1.2726459503173828, 1.143277883529663, 1.031258225440979]
categorical_accuracy = [0.528124988079071,
  0.5496389865875244,
  0.5928571224212646]
precision =  [0.7836644649505615, 0.6816608905792236, 0.7329721450805664]

# Graficar las curvas de entrenamiento
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Pérdida')
plt.plot(epochs, categorical_accuracy, 'r', label='Precisión categórica')
plt.plot(epochs, precision, 'g', label='Precisión')
plt.title('Curvas de entrenamiento')
plt.xlabel('Época')
plt.legend()
plt.show()

In [ ]:
for batch in test_generator:
  X,y = batch
  yhat = model.predict(X)
  pre.update_state(y,yhat)
  acc.update_state(y,yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
scores = model.evaluate_generator(test_generator,steps= len(X_test)//32)